# L8b: Single Index Model Portfolio Optimization
In this lecture, we will explore the Single Index Model (SIM) and its application in portfolio optimization. The SIM simplifies the analysis of asset returns by assuming that the returns of individual assets are influenced by a single common factor, typically the market return. Here, we'll look at using single index models to describe portfolios composed of only risky assets and a combination of risky and risk-free assets.

> __Learning Objectives:__
> 
> By the end of this lecture, you will be able to define and demonstrate mastery of the following key concepts:
> * __Risk and return measures using SIM__ involve computing portfolio expected return and variance using single index model parameters ($\alpha_i$, $\beta_i$) rather than full covariance matrices, simplifying calculations while capturing market-driven correlations.
> * __Risky asset portfolio optimization with SIM__ applies the minimum variance framework using SIM-derived covariances, reducing estimation requirements from $O(N^2)$ to $O(N)$ parameters while maintaining portfolio optimization capabilities.
> * __Risk-free and risky asset portfolios using SIM__ extend the optimization to include zero-variance assets such as treasury securities, formulating the minimum variance problem with combined risky and risk-free asset weights.

Let's get started!
___

## Examples

> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation. Bootstrapping allows us to assess the variability of our estimates and construct confidence intervals, providing insights into the reliability of our model parameters, using simulation techniques.
___

## Concept Review: Single Index Model (SIM)
The Single Index Model (SIM) is a simplified approach to understanding the relationship between the returns of individual assets and the overall market. It assumes that the return of an asset can be expressed as a linear function of the market return plus a unique component specific to the asset.


> __Single Index Model (SIM)__
> 
> Single index models are factor models that consider only the return (growth) of the market factor. These models were originally developed by Sharpe, 1963: [Sharpe, William F. (1963). "A Simplified Model for Portfolio Analysis". Management Science, 9(2): 277-293. doi:10.1287/mnsc.9.2.277.](https://pubsonline.informs.org/doi/abs/10.1287/mnsc.9.2.277)
>
> Suppose the growth of firm $i$ at time $t$ is denoted by $g^{(t)}_{i}$. Then, the single index model of the return (growth rate) is given by:
> $$
g^{(t)}_{i} = \alpha_{i} + \beta_{i}\;g^{(t)}_{M} + \epsilon^{(t)}_{i}
$$
> where $\alpha_{i}$ is the _idiosyncratic (firm-specific) growth_, $\beta_{i}$ is the component of the growth rate of firm $i$ explained by the market (it is also a measure of risk), $g^{(t)}_{M}$ is the growth rate of the market index at time $t$, and $\epsilon^{(t)}_{i}$ denotes an error model associated with firm $i$ (describes growth rate not captured by the firm or market factors). 



### Simulation: Bootstrapping parameter uncertainty
Finally, an interesting way to explore parameter uncertainty is through a Monte Carlo simulation procedure called bootstrapping.

> __Bootstrapping__ The idea is to use our estimated parameters and error model to generate many __synthetic datasets__, then examine the empirical distribution of parameter estimates that we obtain from re-estimating parameters using these datasets. This approach is called __bootstrapping__ because we are _pulling ourselves up by our bootstraps_, i.e., using our fitted model to generate new data and re-estimate parameters, thereby empirically exploring the sampling distribution of our estimator. 

This allows us to see if the empirical results match our theoretical expectations. Let's look at some pseudocode for how we might implement this simulation.

__Initialization:__ Given the design matrix $\hat{\mathbf{X}}$, the estimated parameters $\hat{\boldsymbol{\theta}}$ and the error variance $\hat{\sigma}^2$ from our observed data, a value for the regularization parameter $\delta\geq{0}$, and the number of samples to generate $K$. 

For each $k = 1, 2, \ldots, K$: __do__:
1. Generate synthetic errors: $\boldsymbol{\varepsilon}^{(k)} \sim \mathcal{N}(\mathbf{0}, \Delta{t}\;\hat{\sigma}^2\;\mathbf{I})$
2. Create synthetic observations: $\mathbf{y}^{(k)} \gets \hat{\mathbf{X}}\;\hat{\boldsymbol{\theta}} + \boldsymbol{\varepsilon}^{(k)}$
3. Estimate parameters from the synthetic observation: $\hat{\boldsymbol{\theta}}^{(k)} \gets \left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}} + \delta\;\mathbf{I}\right)^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y}^{(k)}$

Analyze the empirical distribution of $\left\{\hat{\boldsymbol{\theta}}^{(1)}, \hat{\boldsymbol{\theta}}^{(2)}, \ldots, \hat{\boldsymbol{\theta}}^{(K)}\right\}$. 

The empirical mean and covariance of the simulated parameter estimates should approximate our theoretical result:
$$
\begin{align*}
\text{Empirical mean} &\approx \mathbf{S}\;\hat{\boldsymbol{\theta}}\\
\text{Empirical covariance} &\approx \Delta{t}\;\hat{\sigma}^2\;\left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}} + \delta\;\mathbf{I}\right)^{-1}
\end{align*}
$$


Let's look at an example where we quantify parameter uncertainty using bootstrap simulation.

> __Example__
>
> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation. Bootstrapping allows us to assess the variability of our estimates and construct confidence intervals, providing insights into the reliability of our model parameters, using simulation techniques.

___

## Risk and Return Measure using Single Index Models
Using the single index model, we can derive expressions for the expected return and risk (variance) of a portfolio composed of multiple assets. Thus, we have a model for these quantities that does not require us to estimate all pairwise covariances directly from data.

> __Why use Single Index Models?__ 
> 
> The SIM dramatically reduces the number of parameters that must be estimated for portfolio optimization. 
> * For a traditional portfolio of $N$ assets, the traditional approach requires estimating $N$ expected returns and $N(N+1)/2$ unique covariances, a total of $N(N+3)/2$ parameters. 
> * A SIM portfolio requires only $2N$ parameters ($\alpha_i$ and $\beta_i$ for each asset) plus two market parameters ($\mathbb{E}[g_M]$ and $\sigma_m^2$), totaling $2N+2$ parameters. 
> 
> For large portfolios, this reduction is substantial: a 100-asset portfolio requires 5,150 traditional parameters versus only 202 SIM parameters. This parsimony reduces estimation error and improves out-of-sample portfolio performance.

### Reward Measure
Suppose we have a portfolio $\mathcal{P}$ composed of $N$ assets, with weights (dollar fraction) $w_i$ for each asset $i$. The expected annualized growth rate (return) of the portfolio (reward measure) can be expressed as the weighted sum of the expected returns of the individual assets:
$$
\begin{align*}
\mathbb{E}[g_{\mathcal{P}}] &= \sum_{i=1}^{N} w_i\;\mathbb{E}[g_i]\\
&= \sum_{i=1}^{N} w_i\;\left(\alpha_i + \beta_i\;\mathbb{E}[g_M]\right)\\
&= \underbrace{\sum_{i=1}^{N} w_i\;\alpha_i}_{\;\alpha_{\mathcal{P}}} + \underbrace{\left(\sum_{i=1}^{N} w_i\;\beta_i\right)}_{\;\beta_{\mathcal{P}}}\;\mathbb{E}[g_M]\\
&= \alpha_{\mathcal{P}} + \beta_{\mathcal{P}}\;\mathbb{E}[g_M]\quad\blacksquare
\end{align*}
$$

### Risk Measure
The risk of a portfolio is typically measured by the variance of its growth rate (returns). 
The variance of the growth rate (return) of portfolio $\mathcal{P}$ is given by:
$$
\begin{align*}
\operatorname{Var}(g_{\mathcal{P}}) &= 
\sum_{i\in\mathcal{P}}\sum_{j\in\mathcal{P}}w_{i}w_{j}
\operatorname{Cov}\left(g_{i},g_{j}\right)
\end{align*}
$$
where $w_{\star}$ denotes the fraction of asset $\star$ in the portfolio $\mathcal{P}$, and 
$\operatorname{Cov}\left(g_{i},g_{j}\right)$ denotes the covariance between the growth rate of assets $i$ and $j$. Substituting the single index model for the growth rate of asset $i$ and $j$ into the covariance expression (and simplifing) gives:
$$
\begin{align*}
\operatorname{Cov}(g_{i}, g_{j}) = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases}
\end{align*}
$$
where $\sigma_{m}^2$ denotes the variance of the growth rate (return) of the market factor, 
and $\sigma_{\epsilon_{i}}^{2}$ denotes the variance of the firm-specific error model.

#### Error model variance estimation
We assumed the error model $\boldsymbol{\varepsilon} \sim \mathcal{N}(\mathbf{0},\Delta{t}\;\sigma^{2}\;\mathbf{I})$, where $\sigma^{2}$ is the variance of the error model. Of course, we don't know $\sigma^{2}$, so we need to estimate it from the data. A common approach is to estimate the variance of the residuals:
$$
\begin{align*}
\hat{\sigma}^{2} &= \frac{1}{\Delta{t}(n-p)}\;\lVert~\underbrace{\mathbf{y} - \hat{\mathbf{X}}\;\hat{\boldsymbol{\theta}}}_{\text{residual}\;\mathbf{r}}~\rVert^{2}_{2}
\end{align*}
$$
where $n$ is the number of training examples, $p = 2$ is the number of model parameters (including the intercept), and $\hat{\boldsymbol{\theta}}$ is the estimated parameter vector (the $\alpha_{i}$ and $\beta_{i}$ values for asset $i$).
___

## Risky Asset Portfolios using Single Index Models
The risky portfolio allocation problem for a portfolio $\mathcal{P}$ using the single index model is:
$$
\begin{align*}
\text{minimize}~\operatorname{Var}(g_{\mathcal{P}}) &= \sum_{i\in\mathcal{P}}\sum_{j\in\mathcal{P}}w_{i}w_{j}
\operatorname{Cov}\left(g_{i},g_{j}\right) \\
\text{subject to}~\mathbb{E}(g_{\mathcal{P}})& = \alpha_{\mathcal{P}}+\beta_{\mathcal{P}}\;\mathbb{E}(g_{M}) = {R^{*}} \\
\alpha_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\alpha_{i}\\
\beta_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\beta_{i} \\
\sum_{i\in\mathcal{P}}w_{i} & = 1 \\
w_{i}&\geq 0 \qquad \forall{i}\in\mathcal{P} \\
\end{align*}
$$
where the covariance between assets $i$ and $j$ is given by:
$$
\begin{align*}
\operatorname{Cov}(g_{i}, g_{j}) & = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases} \\
\end{align*}
$$

Let's look at an example of optimizing a risky asset portfolio using the single index model, and compare it to the same assets optimized using the full covariance matrix and expected returns estimated from historical data.

> __Example__
>
> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation. Bootstrapping allows us to assess the variability of our estimates and construct confidence intervals, providing insights into the reliability of our model parameters, using simulation techniques.

___

<div>
    <center>
        <img src="figs/Fig-MinVar-RiskFree-Schematic.svg" width="680"/>
    </center>
</div>

## Risky and Risk-Free Asset Portfolios using Single Index Models
We can also consider portfolios that include both risky assets and a risk-free asset. The risk-free asset has a known growth rate (return) $g_f$ and zero variance. For our example, we'll assume a zero-coupon bond, e.g., a STRIPS bond, that matures in some specified time horizon.

The minimum variance portfolio problem for a portfolio $\mathcal{P}$ using the single index model with a combination of risky and risk-free asses is given by:
$$
\begin{align*}
\text{minimize}~\operatorname{Var}(g_{\mathcal{P}}) &= \sum_{i\in\mathcal{P}}\sum_{j\in\mathcal{P}}w_{i}w_{j}
\operatorname{Cov}\left(g_{i},g_{j}\right) \\
\text{subject to}~\mathbb{E}(g_{\mathcal{P}})& = w_{f}g_{f}+\alpha_{\mathcal{P}}+\beta_{\mathcal{P}}\;\mathbb{E}(g_{M}) = {R^{*}}\\
\alpha_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\alpha_{i}\\
\beta_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\beta_{i} \\
w_{f}+\sum_{i\in\mathcal{P}}w_{i} & = 1 \\
w_{f}&\geq{0}\\
w_{i}&\geq{0}\qquad{\forall{i}\in\mathcal{P}}
\end{align*}
$$
where the covariance between assets $i$ and $j$ is given by:
$$
\begin{align*}
\operatorname{Cov}(g_{i}, g_{j}) & = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases} \\
\end{align*}
$$

The terms $w_{i}\geq{0}$ denote the fraction of risky asset $i\in\mathcal{P}$, 
the quantity $w_{f}$ denotes the fraction of risk-free assets in the portfolio, 
$r_{f}$ denotes the risk free rate or return, and $R^{*}$ is the minimum required growth rate (return) 
for overall portfolio $\mathcal{P}$.

Let's look at an example of optimizing a risky and risk-free asset portfolio using the single index model, and compare it to the same assets optimized using the full covariance matrix and expected returns estimated from historical data.

> __Example__
>
> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation. Bootstrapping allows us to assess the variability of our estimates and construct confidence intervals, providing insights into the reliability of our model parameters, using simulation techniques.

___

## Summary
In this lecture, we applied the single index model to portfolio optimization, demonstrating how SIM simplifies the multiasset allocation problem while preserving essential risk-return relationships.

> __Key Takeaways:__
> 
> * __SIM-based risk and return measures:__ We derived portfolio expected return as the sum of portfolio alpha and portfolio beta times market expected return, and variance using SIM parameters, reducing computational complexity from estimating N(N+1)/2 covariances to N beta values and a single market variance.
> * __Risky portfolio optimization:__ We formulated the minimum variance problem using SIM-derived covariances, where the covariance between two different assets equals the product of their betas times market variance, enabling efficient portfolio construction with fewer estimation parameters and reduced estimation error.
> * __Combined risky and risk-free portfolios:__ We extended the optimization framework to include risk-free assets with known return and zero variance, formulating the minimum variance problem that optimizes allocation between treasury securities and risky market portfolios.

In the next lecture, we will explore advanced portfolio reallocation strategies.
___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___